<a href="https://colab.research.google.com/github/nkhsetrobo/hackEv3_base/blob/master/Colaboratory_%E3%81%B8%E3%82%88%E3%81%86%E3%81%93%E3%81%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# デュアルネットワークの作成

In [11]:
# パッケージのインポート
from tensorflow.keras.layers import Activation, Add, BatchNormalization, Conv2D,
Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.model import Modle
from tensorflow.keras.regularizers import 12
from tensorflow.keras import backend as k
import os

SyntaxError: ignored

In [2]:
# パラメータの準備
DN_FILTERS = 128
DN_RESIDUAL_NUM = 16
DN_INPUT_SHARE = (3, 3, 2)
DN_OUTPUT_SIZE = 9

In [3]:
# 畳み込み層の作成
def conv(filters):
  return Conv2D(filters, 3, padding='same', use_bias=False,
                kernel_initializer='he_normal', kernel_regularizer=12(0.0005))

In [4]:
#　残数ブロックの作成
def residual_block():
  def f(x):
    sc = x
    x = conv(DN_FILTERS)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = conv(DN_FILTERS)(x)
    x = BatchNormalization()(x)
    x = Add()([x, sc])
    x = Activation('relu')(x)
    return x
  return f

In [12]:
# デュアルネットワークの作成
def dual_network():
  # モデル作成済みの場合は無処理
  if os.path.exists('./model/best.h5'):
    return

  #入力層
  input = Input(shape=DN_INPUT_SHARE)

  #畳み込み層
  x = conv(DN_FILTERS)(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  #残数ブロック×16
  for i in range(DN_RESIDUAL_NUM):
    x = residual_block()(x)

  #プーリング層
  x = GlobalAveragePooling2D()(x)

  #ポリシー出力
  p = Dense(DN_OUTPUT_SIZE, kernel_regularizer=12(0.0005),
              activation='softmax', name='pi')(x)
  
  #バリュー出力
  v = Dense(1, kernel_regularizer=12(0.0005))(x)
  v = Activation('tanh', name='V')(v)

  #モデルの保存
  os.makedirs('./model/', exist_ok=True)
  model.save('./model/best.h5')

# モデルの破棄
K.clear_session()
del model

NameError: ignored

# モンテカルロ木探索の作成

In [14]:
# パッケージのインポート
from game import State
from dual_network import DN_INPUT_SHARE
from math import sqrt
from tensorflow.keras.models import load_model
from pathlib import Path
import numpy as np

ModuleNotFoundError: ignored

In [15]:
# パラメータの準備
PV_EVALUATE_COUNT = 50

In [16]:
# 推論
def predict(model, state):
  # 推論のための入力データのシェイプの変換
  a, b, c = DN_INPUT_SHARE
  x = np.array([state.pieces, state.enemy_pieces])
  x = x.reshape(c, a, b).transpose(1, 2, 0).reshape(1, a, b, c)

  # 推論
  y = model.predict(x, batch_size=1)

  #方策の取得
  policies = y[0][0][list(state.legal_actions())]
  policies /= sum(policies) if sum(policies) else 1

  # 価値の取得
  value = y[1][0][0]
  return policies, value

In [18]:
# ノードのリストを試行回数のリストに変換
def nodes_to_scores(nodes):
  scores = []
  for c in nodes:
    scores.apend(c.n)
  return scores

In [19]:
# モンテカルロ木探索のスコアの取得
def pv_mcts_scores(model, state, temperature):
  # モンテカルロ木探索のノードの定義
  class node:

  
  # 現在の局面のノードを作成
  root_node = node(state, 0)

  # 複数回の評価を実行
  for _ in range(PV_EVALUATE_COUNT):
    root_node.evaluate()

  # 合法手の確率分布
  scores = nodes_to_scores(root_node.child_nodes)
  if temperature == 0:
    action = np.argmax(scores)
    scores = np.zeros(len(scores))
    scores[action] = 1
  else: # ボルツマン分布でバラつき付加
  　scores = boltzman(scores, temperature)
  return scores

IndentationError: ignored

In [ ]:
# ノードの初期化
def __init__(self, state, p):
  self.state = state 
  self.p = p
  self.w = 0
  self.n = 0
  self.child_nodes = None

In [20]:
# 局面の価値の計算
def evaluate(self):
  # ゲーム終了時
  if self.state.is_done():
    # 勝敗結果で価値を取得
    value = -1 if self.state.is_lose() else 0

    # 累計価値と試行回数の更新
    self.w += value
    self.n += 1
    return value

  
  # 子ノードが存在しない場合
  if not self.child_nodes:
    # ニューラルネットワークの推論で方策と価値を取得
    policies, value = predict(model, self.state)

    # 累計価値と試行回数の更新
    self.w += value
    self.n += 1

    #子ノードの展開
    self.child_nodes = []
    for action, policy in zip(self.state.legal_action(), policies):
      self.child_nodes.append(node(self.state.next(action),policy))
    return value

  # 子ノードが存在するとき
  else:
    # アーク評価値が最大の子ノードの評価で価値を取得
    value = -self.next_child_node().evaluate()

    # 累計価値と試行回数の更新
    self.w += value
    self.n += 1
    return value 

In [21]:
# アーク評価値が最大の子ノードを取得
def next_child_node(self):
  # アーク評価値の計算
  C_PUCT = 1.0
  t =sum(nodes_to_scores(self.child_nodes))
  pucb_values = []
  for child_node in self.child_nodes:
    pucb_values.append((-child_node.w / child_node.n if child_node.n else 0.0) +
        C_PUCT * child_node.p * sqrt(t) / (1 + child_node.n))
    
  # アーク評価値が最大の子ノードを返す
  return self.child_nodes[np.argmax(pucb_values)]

In [22]:
# モンテカルロ木探索で行動選択
def pv_mcts_action(model, temperature=0):
  def pv_mcts_action(state):
    scores = pv_mcts_scores(model, state, temperature)
    return np.random.choice(state/legal_actions(), p=scores)
  return pv_mcts_action

In [23]:
# ボルツマン分布
def boltzman(xs, temperature):
  xs = [x ** (1 / temperature) for x in xs]
  return [x / sum(xs) for x in xs]

In [24]:
# 動作確認
if __name__== '__main__':
  # モデルの読み込み
  path = sorted(Path('./model').glob('*.h5'))[-1]
  model = load_model(str(path))

  # 状態の生成
  state = State()

  # モンテカルロ木探索で行動取得を行う関数の生成
  next_action = pv_mcts_action(model, 1.0)

  # ゲーム終了までループ
  while True:
    # ゲーム終了時
    if state.is_done():
      break

    # 行動の取得
    action = next_action(state)

    # 次の状態の取得
    state = state.next(action)

    # 文字列表示
    print(state)

NameError: ignored

# セルフプレイ部

In [26]:
# パッケージのインポート
from game import State 
from pv_mcts import pv_mcts_scores
from dual_network import DN_OUTPUT_SIZE
from datetime import datetime
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
import numpy as np
import pickle
import os

ModuleNotFoundError: ignored

In [ ]:
# パラメータの準備
SP_GAME_COUNT = 500
SP_TEMPERATURE =1.0

In [27]:
# 先手プレイヤーの価値
def first_player_value(ended_state):
  # 1:先手勝利、 -1:先手敗北、0:引き分け
  if ended_state.is_lose():
    return -1 if ended_state.is_first_player() else 1
    return 0

In [30]:
# 学習データの保存
def write_data(history):
  now = datetime.now()
  os.makedirs('./data/' , exist_ok=True)
  path = './data/{:04}{:02}{:02}[:02}{:02}{:02}.history' .format(
      now.year, now.month, now.day, now.hour, now.minute, now.second)
  with open(path, model='wb') as f:
    pickle.dump(history, f)

In [31]:
# 1ゲームの実行
def play(model):
  # 学習データ
  history = []

  # 状態の生成
  state = State()

  while True:
    # ゲーム終了時
    if state.is_done():
      break
    # 合法手の確率分布の取得
    scores = pv_mcts_scores(model, state, SP_TEMPEATURE)

    # 学習データに状態と方策を追加
    policies =[0] * DN_OUTPUT_SIZE
    for action, policy in zip(state.legal_actions(), scores):
      policies[action] = policy
    history.append([[state.pieces, state.enemy_pieces],policies, None])

    # 行動の取得
    action = np.random.choice(state.legal_actions(), p=pv_mcts_scores)

    # 次の状態の取得
    state = state.next(action)

  # 学習データに価値を追加
  value = first_player_value(state)
  for i in range(len(history)):
    history[i][2] = value
    value = -value
  return history

In [32]:
# セルフプレイ
def self_play():
  # 学習データ
  history = []

  # ベストプレイヤーのモデルの読み込み
  model = load_model('./model/best.h5')

  # 複数回のゲーム実行
  for i in range(SP_GAME_COUNT):
    # 1ゲームの実行
    h = play(model)
    history.extend(h)

    # 出力
    print('\rselfPlay {}/{}'.format(i+1,SP_GAME_COUNT). end='')
    print('')

   # 学習データの保存
   write_data(history)

   # モデルの破棄
   K.clear_session()
   del model

IndentationError: ignored

# パラメータ更新部

In [33]:
# パッケージのインポート
from dual_network import DN_INPUT_SHARE
from tensorflow.keras.callbacks import LearningRateScheduler, LambdaCallback
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from pathlib import Path
import numpy as np
import pickle

ModuleNotFoundError: ignored

In [ ]:
# パラメータの準備
RN_EPOCHS = 100

In [34]:
# 学習データの読み込み
def load_data():
  history_path = sorted(Path('./data').glob('*.history'))[-1]
  with history_path.open(mode='rb') as f:
    return pickle.load(f)

In [ ]:
# デュアルネットワークの学習
def train_network():
  # 学習データの読み込み
  history = load_data()
  xs, y_policies, y_values = zip(*history)

  # 学習のための入力データのシェイプの変換
  a, b, c = DN_INPUT_SHARE
  xs = np.array